In [2]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

In [58]:
url = 'https://airkorea.or.kr/web/'
driver = webdriver.Chrome('chromedriver.exe')
driver.get(url)

In [59]:
#팝업닫기(팜업이 2개면 2번 실행)
driver.switch_to_window(driver.window_handles[1])
driver.find_element(By.ID, 'pop').click()

C:\Users\user\AppData\Local\Temp\ipykernel_11016\296760026.py:2: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[1])


In [60]:
#인터넷창 전환
driver.switch_to_window(driver.window_handles[0])
#실시간 자료조회
driver.find_element(By.XPATH, '//*[@id="topMenu"]/li[2]/a').click()

C:\Users\user\AppData\Local\Temp\ipykernel_11016\954089337.py:2: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[0])


In [61]:
#시작일 시간선택
driver.find_element(By.XPATH, '//*[@id="from_date_hour"]/option[1]').click()
#종료일 시간선택
driver.find_element(By.XPATH, '//*[@id="to_date_hour"]/option[1]').click()

In [62]:
#시작일 달력선택
driver.find_element(By.CLASS_NAME, 'ui-datepicker-trigger').click()
#17년도 선택
driver.find_element(By.XPATH, '//*[@id="ui-datepicker-div"]/div[1]/div/select[1]/option[2]').click()
#12월 선택
driver.find_element(By.XPATH, '//*[@id="ui-datepicker-div"]/div[1]/div/select[2]/option[12]').click()
#1일 선택
driver.find_element(By.CLASS_NAME, 'ui-state-default').click()

In [63]:
#종료일 달력선택
driver.find_element(By.CLASS_NAME, 'W100.MgL10.calendar.hasDatepicker').click()
#18년도 선택
driver.find_element(By.XPATH, '//*[@id="ui-datepicker-div"]/div[1]/div/select[1]/option[3]').click()
#1월 선택
driver.find_element(By.XPATH, '//*[@id="ui-datepicker-div"]/div[1]/div/select[2]/option[1]').click()
#1일 선택
driver.find_element(By.CLASS_NAME, 'ui-state-default').click()

In [4]:
df_addr = pd.read_excel('./관측소주소.xlsx')

TypeError: expected <class 'str'>

In [65]:
#기상관측소 검색해서 다시 주소 정리 부탁...
df_addr

,관측소명,주소
0,백령도(102),인천광역시 옹진군 백령면 진촌리
1,강화(201),인천광역시 강화군 불은면 삼성리
2,파주(99),경기도 파주시 문산읍 마정로
3,철원(95),강원도 철원군 갈말읍 명성로
4,동두천(98),경기도 동두천시 방죽로 16-47
...,...,...
97,제주(184),제주특별자치도 제주시 만덕로 6길 32 제주지방기상청
98,성산포(265),NaN
99,서귀포(189),제주특별자치도 서귀포시 태평로 439번길
100,고산(185),제주특별자치도 제주시 한경면 노을해안로


In [67]:
df_addr['주소'][0] = '인천광역시 옹진군 백령면 백령로 336'

In [68]:
#주소검색창 열기
driver.find_element(By.CLASS_NAME, 'W250').click()

In [69]:
#인터넷창 전환(주소검색창)
driver.switch_to_window(driver.window_handles[1])  
# driver.get_window_position(driver.window_handles[1])
time.sleep(1)
#주소 입력
driver.find_element(By.CLASS_NAME, 'popSearchInput').send_keys(df_addr['주소'][0])
#검색
driver.find_element(By.XPATH, '//*[@id="searchContentBox"]/div[1]/fieldset/span/input[2]').click()
#주소선택
driver.find_element(By.ID, 'roadAddrDiv1').click()
#주소입력
driver.find_element(By.CLASS_NAME, 'btn-bl').click()

C:\Users\user\AppData\Local\Temp\ipykernel_11016\2810566663.py:2: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[1])


In [70]:
#인터넷창 전환(자료조회)
driver.switch_to_window(driver.window_handles[0])  
#주소검색
driver.find_element(By.XPATH, '//*[@id="cont_body"]/form[2]/div[1]/a[2]').click()

C:\Users\user\AppData\Local\Temp\ipykernel_11016\2244936966.py:2: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[0])


In [71]:
#측정자료가 0이면 다른 주소로 재검색
soup = BeautifulSoup(driver.page_source, 'html.parser')
addr = soup.select('.st_1.tr_hover > tbody > tr')
count = 0
while True:
    lis = soup.select_one('.st_1.stoke > tbody > tr')
    values = lis.select('td')
    addr_num = 1
    for i in range(1, len(values), 2):
        if values[i].text == '-':
            count += 1
    if count == 6:
        count = 0
        driver.find_element(By.ID, f'stationCode{addr_num}').click()
        driver.find_element(By.CLASS_NAME, 'search.MgL10').click()
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        addr_num += 1
        if addr_num == len(addr):
            break
    else:
        break

In [74]:
#엑셀 다운로드(17년12월1~18년1월1일)
driver.find_element(By.CLASS_NAME, 'xls').click()

In [76]:
#18년1월1일~23년1월1일
for year in range(3,8): #3: 18년, 7: 22년
    end_year = year
    for month in range(1,13):
        #시작일 달력선택
        driver.find_element(By.CLASS_NAME, 'ui-datepicker-trigger').click()
        #년도 선택
        driver.find_element(By.XPATH, f'//*[@id="ui-datepicker-div"]/div[1]/div/select[1]/option[{year}]').click()
        #월 선택
        driver.find_element(By.XPATH, f'//*[@id="ui-datepicker-div"]/div[1]/div/select[2]/option[{month}]').click()
        #1일 선택
        driver.find_element(By.CLASS_NAME, 'ui-state-default').click()
        time.sleep(1)
        
        #종료일 연도와 달 지정
        end_month = month + 1
        if end_month == 13:
            end_year = year + 1 
            end_month = 1
        #종료일 달력선택
        driver.find_element(By.CLASS_NAME, 'W100.MgL10.calendar.hasDatepicker').click()
        #년도 선택
        driver.find_element(By.XPATH, f'//*[@id="ui-datepicker-div"]/div[1]/div/select[1]/option[{end_year}]').click()
        #월 선택
        driver.find_element(By.XPATH, f'//*[@id="ui-datepicker-div"]/div[1]/div/select[2]/option[{end_month}]').click()
        #일 선택
        driver.find_element(By.CLASS_NAME, 'ui-state-default').click()
        time.sleep(1)
        
        #검색
        driver.find_element(By.CLASS_NAME, 'search.MgL10').click()
        time.sleep(3)
        
        #엑셀 다운로드
        driver.find_element(By.CLASS_NAME, 'xls').click()
        time.sleep(2)

In [ ]:
# 하나의 주소에 대한 데이터 다운로드가 끝나면 다운로드 폴더에 있는 61개의 파일을 'ㅇㅇ 관측소' 폴더를 만들어서 넣어두면 될듯? 
# 다 끝나면 파일 묶는것도 해주면 감사~